Importing Libraries

In [152]:
import pandas as pd


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import mlflow


In [153]:
file_path='/Users/mohamedafrith/Downloads/playground-series-s4e11/train.csv'
df=pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Profession                             104070 non-null  object 
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

In [154]:
for i in df.columns:
    unique=df[i].nunique()
    print(i,unique)

id 140700
Name 422
Gender 2
Age 43
City 98
Working Professional or Student 2
Profession 64
Academic Pressure 5
Work Pressure 5
CGPA 331
Study Satisfaction 5
Job Satisfaction 5
Sleep Duration 36
Dietary Habits 23
Degree 115
Have you ever had suicidal thoughts ? 2
Work/Study Hours 13
Financial Stress 5
Family History of Mental Illness 2
Depression 2


In [155]:
df.drop('Profession', axis=1, inplace=True)

In [156]:
df=df.fillna(0)

In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 19 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Academic Pressure                      140700 non-null  float64
 7   Work Pressure                          140700 non-null  float64
 8   CGPA                                   140700 non-null  float64
 9   Study Satisfaction                     140700 non-null  float64
 10  Job Satisfaction                       140700 non-null  

In [158]:
list_sd=list(df['Sleep Duration'].unique())
list_sd

['More than 8 hours',
 'Less than 5 hours',
 '5-6 hours',
 '7-8 hours',
 'Sleep_Duration',
 '1-2 hours',
 '6-8 hours',
 '4-6 hours',
 '6-7 hours',
 '10-11 hours',
 '8-9 hours',
 '40-45 hours',
 '9-11 hours',
 '2-3 hours',
 '3-4 hours',
 'Moderate',
 '55-66 hours',
 '4-5 hours',
 '9-6 hours',
 '1-3 hours',
 'Indore',
 '45',
 '1-6 hours',
 '35-36 hours',
 '8 hours',
 'No',
 '10-6 hours',
 'than 5 hours',
 '49 hours',
 'Unhealthy',
 'Work_Study_Hours',
 '3-6 hours',
 '45-48 hours',
 '9-5',
 'Pune',
 '9-5 hours']

In [159]:
# Function to categorize sleep duration
def categorize_sleep(duration):
    try:
        if isinstance(duration, str):
            duration = duration.lower()  # Convert to lowercase for uniformity
            
            if any(keyword in duration for keyword in ['1-2', '2-3', '3-4', '4-5', 'less than 5', 'than 5', '1-6']):
                return 1  # 0-5 hours (Short Sleep)
            elif any(keyword in duration for keyword in ['5-6', '6-7', '6-8', '5-7', '4-6', '3-6']):
                return 1  # 5-7 hours (Still Short Sleep)
            elif any(keyword in duration for keyword in ['7-8', '8 hours', '8-9']):
                return 2  # 7-9 hours (Optimal Sleep)
            elif any(keyword in duration for keyword in ['9+', '9-11', '10-11', 'more than 8', '9-6', '9-5', '10-6']):
                return 3  # 9+ hours (Long Sleep)
        
        return None  # If not a sleep duration value, return None
    except:
        return None

# Apply categorization to the list

df['Sleep Duration'] = df['Sleep Duration'].apply(categorize_sleep)




In [160]:
df['Sleep Duration'].unique()

array([ 2.,  1., nan,  3.])

In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 19 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Academic Pressure                      140700 non-null  float64
 7   Work Pressure                          140700 non-null  float64
 8   CGPA                                   140700 non-null  float64
 9   Study Satisfaction                     140700 non-null  float64
 10  Job Satisfaction                       140700 non-null  

In [162]:
df.isna().sum()


id                                        0
Name                                      0
Gender                                    0
Age                                       0
City                                      0
Working Professional or Student           0
Academic Pressure                         0
Work Pressure                             0
CGPA                                      0
Study Satisfaction                        0
Job Satisfaction                          0
Sleep Duration                           18
Dietary Habits                            0
Degree                                    0
Have you ever had suicidal thoughts ?     0
Work/Study Hours                          0
Financial Stress                          0
Family History of Mental Illness          0
Depression                                0
dtype: int64

In [163]:
df = df.fillna(method='ffill')


/var/folders/pk/yv119hjs3_34653qn5_6djlh0000gn/T/ipykernel_51316/3984534036.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


In [164]:
df[['Age', 'Academic Pressure',
    'Work Pressure','Study Satisfaction',
    'Job Satisfaction','Sleep Duration',
    'Financial Stress']] = df[['Age', 'Academic Pressure','Work Pressure','Study Satisfaction'
                               ,'Job Satisfaction','Sleep Duration','Financial Stress']].astype(int)


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 19 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  int64  
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Academic Pressure                      140700 non-null  int64  
 7   Work Pressure                          140700 non-null  int64  
 8   CGPA                                   140700 non-null  float64
 9   Study Satisfaction                     140700 non-null  int64  
 10  Job Satisfaction                       140700 non-null  

In [166]:
df['Work/Study Hours']=df['Work/Study Hours'].astype(int)

In [167]:
df.drop(columns=['id','Name','City','Degree'], axis=1, inplace=True)


In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 15 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Gender                                 140700 non-null  object 
 1   Age                                    140700 non-null  int64  
 2   Working Professional or Student        140700 non-null  object 
 3   Academic Pressure                      140700 non-null  int64  
 4   Work Pressure                          140700 non-null  int64  
 5   CGPA                                   140700 non-null  float64
 6   Study Satisfaction                     140700 non-null  int64  
 7   Job Satisfaction                       140700 non-null  int64  
 8   Sleep Duration                         140700 non-null  int64  
 9   Dietary Habits                         140700 non-null  object 
 10  Have you ever had suicidal thoughts ?  140700 non-null  

In [169]:
df['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [170]:
gender_mapping = {'Male': 0, 'Female': 1}
df['Gender'] = df['Gender'].map(gender_mapping)


In [171]:
df['Working Professional or Student'].unique()

array(['Working Professional', 'Student'], dtype=object)

In [172]:
profession_mapping = {'Working Professional': 0, 'Student': 1}
df['Working Professional or Student'] = df['Working Professional or Student'].map(profession_mapping)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 15 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Gender                                 140700 non-null  int64  
 1   Age                                    140700 non-null  int64  
 2   Working Professional or Student        140700 non-null  int64  
 3   Academic Pressure                      140700 non-null  int64  
 4   Work Pressure                          140700 non-null  int64  
 5   CGPA                                   140700 non-null  float64
 6   Study Satisfaction                     140700 non-null  int64  
 7   Job Satisfaction                       140700 non-null  int64  
 8   Sleep Duration                         140700 non-null  int64  
 9   Dietary Habits                         140700 non-null  object 
 10  Have you ever had suicidal thoughts ?  140700 non-null  

In [173]:
df['Study Satisfaction']=df['Study Satisfaction'].astype(int)

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 15 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Gender                                 140700 non-null  int64  
 1   Age                                    140700 non-null  int64  
 2   Working Professional or Student        140700 non-null  int64  
 3   Academic Pressure                      140700 non-null  int64  
 4   Work Pressure                          140700 non-null  int64  
 5   CGPA                                   140700 non-null  float64
 6   Study Satisfaction                     140700 non-null  int64  
 7   Job Satisfaction                       140700 non-null  int64  
 8   Sleep Duration                         140700 non-null  int64  
 9   Dietary Habits                         140700 non-null  object 
 10  Have you ever had suicidal thoughts ?  140700 non-null  

In [175]:
df['Dietary Habits'].value_counts()


Dietary Habits
Moderate             49705
Unhealthy            46227
Healthy              44741
0                        4
Yes                      2
No                       2
More Healthy             2
Class 12                 1
Indoor                   1
Male                     1
Vegas                    1
M.Tech                   1
Less Healthy             1
No Healthy               1
Hormonal                 1
Electrician              1
1.0                      1
Mihir                    1
Less than Healthy        1
3                        1
Gender                   1
BSc                      1
Pratham                  1
2                        1
Name: count, dtype: int64

In [176]:
# Define valid categories
valid_categories = ['Moderate', 'Unhealthy', 'Healthy']

# Find mode of the column
mode_value = df['Dietary Habits'].mode()[0]

# Replace invalid values with mode
df['Dietary Habits'] = df['Dietary Habits'].apply(lambda x: x if x in valid_categories else mode_value)


In [177]:
diet_mapping={'Healthy':0, 'Unhealthy':1, 'Moderate':2}
df['Dietary Habits'] = df['Dietary Habits'].map(diet_mapping)

In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 15 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Gender                                 140700 non-null  int64  
 1   Age                                    140700 non-null  int64  
 2   Working Professional or Student        140700 non-null  int64  
 3   Academic Pressure                      140700 non-null  int64  
 4   Work Pressure                          140700 non-null  int64  
 5   CGPA                                   140700 non-null  float64
 6   Study Satisfaction                     140700 non-null  int64  
 7   Job Satisfaction                       140700 non-null  int64  
 8   Sleep Duration                         140700 non-null  int64  
 9   Dietary Habits                         140700 non-null  int64  
 10  Have you ever had suicidal thoughts ?  140700 non-null  

In [179]:
sucide_mapping={'No':0, 'Yes':1}
df['Have you ever had suicidal thoughts ?']=df['Have you ever had suicidal thoughts ?'].map(sucide_mapping)


In [180]:
def mapping(column_name):
    unique_values = df[column_name].unique()
    mapping_dict = {val: idx for idx, val in enumerate(unique_values)}
    df[column_name] = df[column_name].map(mapping_dict)



In [181]:
mapping('Family History of Mental Illness')

In [182]:
X = df.drop(columns='Depression', axis=1)
y = df[['Depression']]


In [183]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1️⃣ Split data first
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2️⃣ Scale only on training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit & transform on training data
X_test_scaled = scaler.transform(X_test)  # Only transform test data (No fit!)

# Now, train your model using X_train_scaled


In [184]:
from sklearn.preprocessing import StandardScaler
import pickle

# Initialize the scaler
scaler = StandardScaler()

# Fit on training data only (not the whole dataset!)
scaler.fit(X_train)

# Save the trained scaler
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)



In [185]:
import torch

# Convert X_train and X_test (assuming they are already NumPy arrays)
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32)

# Convert y_train and y_test from DataFrame to NumPy array first
y_train_torch = torch.tensor(y_train.to_numpy(), dtype=torch.float32).reshape(-1, 1)
y_test_torch = torch.tensor(y_test.to_numpy(), dtype=torch.float32).reshape(-1, 1)


In [186]:
import torch.nn as nn
 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(14, 7)
        self.relu = nn.ReLU()
        self.output = nn.Linear(7, 1)
        self.sigmoid = nn.Sigmoid()
 
    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x

In [187]:
model=NeuralNetwork()

In [188]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm

def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    n_epochs = 5   # number of epochs to run
    batch_size = 64  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

In [189]:
model_train(model,X_train_torch,y_train_torch,X_test_torch,y_test_torch)


0.9330490231513977

In [190]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [191]:
# Initialize MLflow experiment
mlflow.set_experiment("Depression_Prediction")

# Train the model and log into MLflow
with mlflow.start_run(run_name='Depression_model'):
    mlflow.log_param("input_size", 14)
    mlflow.log_param("hidden_size", 7)
    mlflow.log_param("output_size", 1)
    

    # Log the PyTorch model in MLflow
    mlflow.pytorch.log_model(model, artifact_path="depression_model")

    print("Model logged in MLflow successfully!")

2025/03/03 17:02:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged in MLflow successfully!
🏃 View run Depression_model at: http://127.0.0.1:5000/#/experiments/389209041133176424/runs/c53db4608bbf4a668de65b844c984a97
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/389209041133176424


In [192]:
import torch
torch.save(model.state_dict(), "model.pth")  # Save model weights only
